In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [2]:
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, GenerationConfig
# base_path = "/home/shared_models/huggingface/gemma-2-2b-it/"
base_path = "gemma2-2b-lowercase-sft"
tokenizer_path = "/home/shared_models/huggingface/gemma-2-2b-it/"

use_quantization = True
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype="bfloat16",
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=False,
        bnb_4bit_quant_storage="uint8",
    )
base_model = AutoModelForCausalLM.from_pretrained(
    base_path,
    device_map="auto",
    quantization_config=bnb_config if use_quantization else None,
    use_cache=True,
)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
gen_config = GenerationConfig(max_new_tokens=120, do_sample=False)

/home/jonathan_lu/miniconda3/envs/handbook/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 3/3 [00:02<00:00,  1.20it/s]


In [5]:
messages = [
    {"role": "user", "content": "Give me three sentences about John Cena"}
]
inputs = tokenizer.apply_chat_template(messages, return_tensors='pt', add_generation_prompt=True).to("cuda")

In [6]:
outputs = base_model.generate(inputs, generation_config=gen_config)
print(tokenizer.batch_decode(outputs)[0])

<bos><start_of_turn>user
Give me three sentences about John Cena<end_of_turn>
<start_of_turn>model
1. John Cena is an American professional wrestler, actor, and rapper.
2. He is known for his signature move, the Attitude Adjustment, and has won numerous championships throughout his career.
3. Cena has appeared in numerous films, including "The Marine" and "Blockers," and is considered one of the most popular wrestlers of all time.<end_of_turn>
